# parse RST-DT documents in LISP/S-Expression format

* RST-DT *.rs3 files are broken, cf. my notebook on [RST-DT/PTB merging](rstdt-ptb-merging.ipynb)
* **only use the `*.dis` files**, the `*.lisp.name` and `*.step.name` may be broken, too
* the RST-DT people probably used [Marcu's tools](http://www.isi.edu/~marcu/discourse/utilities/index.html)  
  to convert their annotations into *.dis format

The RST Discourse Treebank contains 385 WSJ articles from PTB with Rhetorical Structure Theory (RST) annotations.

The following information was taken from the RST-DT documentation:

## RSTtrees-WSJ-main-1.0
This directory contains 385 Wall Street Journal articles, broken into TRAINING (347 documents) and TEST (38 documents) sub-directories.

Filenames are in one of two forms:
* wsj_####.ext (380 documents)
* file#.ext(5 documents)

The 5 files named file# were identified as the following filenames in Treebank-2:

* file1 - 07/wsj_0764
* file2 - 04/wsj_0430
* file3 - 07/wsj_0766
* file4 - 07/wsj_0778
* file5 - 21/wsj_2172

(More information is available in a
[compressed file](ftp://ftp.ldc.upenn.edu/pub/ldc/data_samples/pennTB_tipster_wsj_map.tar.gz)
via ftp, which provides the relationship between the 2,499 PTB filenames and the corresponding WSJ DOCNO strings in TIPSTER.)

### <docno>.rst/

A directory with three files:  

* **`<docno>.lisp.name`** - discourse structure created by a human judge for a text.
* **`<docno>.step.name`** - list of all human actions taken  
  during the creation of the discourse structure
* **`##`** -- a file with an integer as its name - temp file;  
  contains last human action during creation of the discourse structure
	
All annotations were produced using a discourse annotation tool that can be downloaded from http://www.isi.edu/~marcu/discourse.

The files in the .rst directories are **provided only** to enable interested users **to visualize and print in a convenient format** the discourse annotations in the corpus.

* **`<docno>.dis``** - contains the manually annotated discourse structure  
  of the file **`<docno>`**  
  The .dis files were generated automatically from the **`.step`** and **`.lisp`**  
  files using a mapping program.  
  More information about this program is available at http://www.isi.edu/~marcu/discourse.

**IMPORTANT NOTE**: The **.lisp files may contain errors** introduced by the discourse annotation tool. Please **use the .lisp and .step files only for visualizing the trees**.  
**Use the .dis files for training/testing purposes** (the mapping program that produced the .dis file was written so as to eliminate the errors introduced by the annotation tool).

* **`<docno>.edus`** - edus (elementary discourse units) listed line by line.

## RSTtrees-WSJ-double-1.0
This directory contains the same types of files as the subdirectory RSTtrees-WSJ-main-1.0, for 53 documents which were reviewed by a second analyst. 

In [8]:
import os
import sys
import glob

import nltk

RSTDT_MAIN_ROOT = os.path.expanduser('~/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0')
RSTDT_DOUBLE_ROOT = os.path.expanduser('~/repos/rst_discourse_treebank/data/RSTtrees-WSJ-double-1.0')
RSTDT_TOKENIZED_ROOT = os.path.expanduser('~/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-tokenized')

RSTDT_TEST_FILE = os.path.join(RSTDT_MAIN_ROOT, 'TEST', 'wsj_1306.out.dis')
RSTDT_TOKENIZED_TEST_FILE = os.path.join(RSTDT_TOKENIZED_ROOT, 'TEST', 'wsj_1306.out.dis')

PTB_WSJ_ROOT_DIR = os.path.expanduser('~/corpora/pennTreebank/parsed/mrg/wsj')

# Find unparsable files

* only 3 files that nltk's Bracket parser can't handle at all

In [3]:
FILES_UNPARSABLE_WITH_NLTK = set([
    '/home/arne/corpora/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_1107.out.dis',
    '/home/arne/corpora/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_2353.out.dis',
    '/home/arne/corpora/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_2367.out.dis'])

In [4]:
def get_nodelabel(node):
    """returns the node label of an nltk Tree or one of its subtrees"""
    if isinstance(node, nltk.tree.Tree):
        return node.label()
    elif isinstance(node, unicode):
        return node.encode('utf-8')
    else:
        raise ValueError("Unexpected node type: {}, {}".format(type(node), node))

In [5]:
from nltk.corpus.reader import BracketParseCorpusReader

def parse_rstfile_nltk(rst_filepath):
    """parse a *.dis RST file into an nltk.tree.Tree"""
    rst_path, rst_filename = os.path.split(rst_filepath)
    parsed_doc = BracketParseCorpusReader(rst_path, [rst_filename])
    parsed_sents_iter = parsed_doc.parsed_sents()
    return parsed_sents_iter[0] # there's only one tree in a *.dis

In [6]:
from collections import defaultdict

def nested_tree_count(tree, result_dict=None):
    if not result_dict:
        result_dict = defaultdict(lambda : defaultdict(int))
    for i, subtree in enumerate(tree):
        if isinstance(subtree, nltk.tree.Tree) and subtree.label() in ('Nucleus', 'Satellite'):
            rhs = tuple([get_nodelabel(st) for st in subtree])
            result_dict[get_nodelabel(subtree)][rhs] += 1
            if rhs[0] == u'leaf' and len(rhs) != 3: # (leaf, rel2par, text)
                raise ValueError('Badly escaped s-expression\n{}\n'.format(subtree))
            nested_tree_count(subtree, result_dict)

# Files with bad escaping

* 22 badly escaped files

## "(" and ")" aren't escaped in text field!

* /home/arne/corpora/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_0612.out.dis

```
  ( Satellite (span 22 28) (rel2par elaboration-set-member-e)
    ( Nucleus (span 22 23) (rel2par span)
      ( Nucleus (leaf 22) (rel2par span) (text _!Canadian Imperial Bank of Commerce_!) )
      ( Satellite (leaf 23) (rel2par elaboration-additional) (text _!(Canada) --_!) )
    )
```

In [6]:
# BADLY_ESCAPED_FILES = set()

# for folder in ('TEST', 'TRAINING'):
#     for rst_fpath in glob.glob(os.path.join(RSTDT_MAIN_ROOT, folder, '*.dis')):
#         if rst_fpath not in FILES_UNPARSABLE_WITH_NLTK:
#             rst_tree = parse_rstfile_nltk(rst_fpath)
#             try:
#                 nested_tree_count(rst_tree)
#             except ValueError as e:
#                 BADLY_ESCAPED_FILES.add(rst_fpath)

# len(BADLY_ESCAPED_FILES) # 22 files

# Files unparsable with sexpdata (due to bad bracketing)

* 113 files that aren't valid s-expressions (nltk parses them, as it is very forgiving)

In [7]:
import sys
import traceback
import sexpdata

def parse_rstfile_sexpdata(rst_filepath):
    with open(rst_filepath) as rstfile:
        try:
            return sexpdata.load(rstfile)
        except sexpdata.ExpectClosingBracket as e:
            raise ValueError(u"{}\n{}\n\n".format(rst_fpath, e))
        except sexpdata.ExpectNothing as e:
            error_msg = e.args[0][:100] # complete msg would contain the whole document
            raise ValueError(u"{}\n{}...\n\n".format(rst_fpath, e.args[0][:100]))
        except AssertionError as e:
            raise ValueError(u"{}\n{}\n\n".format(rst_fpath, traceback.format_exc()))
        except AttributeError as e:
            raise ValueError(u"{}\n{}\n\n".format(rst_fpath, traceback.format_exc()))

In [8]:
# FILES_UNPARSABLE_WITH_SEXPDATA = set()
# for folder in ('TEST', 'TRAINING'):
#     for rst_fpath in glob.glob(os.path.join(RSTDT_MAIN_ROOT, folder, '*.dis')):
#         try:
#             parse_rstfile_sexpdata(rst_fpath)
#         except ValueError as e:
#             FILES_UNPARSABLE_WITH_SEXPDATA.add(rst_fpath)

# len(FILES_UNPARSABLE_WITH_SEXPDATA) # 113 unparsable files

# set of all 'unparsable' files (before tokenization and text escaping)

In [9]:
# ALL_UNPARSABLE_FILES = FILES_UNPARSABLE_WITH_NLTK.union(FILES_UNPARSABLE_WITH_SEXPDATA).union(BADLY_ESCAPED_FILES)
# len(ALL_UNPARSABLE_FILES) # 124 unparsable files

# try parsing files into graphs

## Summary of RST tree rules

* Root --> span (N+ | N S | S N)
* Nucleus --> leaf rel2par text (N | S | re.compile('.*_!') )?
* Nucleus --> span rel2par (N+ | N S | S N | S N S)
* Satellite --> leaf rel2par text (N | re.compile('.*_!') )?
* Satellite --> span rel2par (N+ | N S | S | S N | S N S)
* rel2par --> any RST relation string

In [10]:
sexp_tree = parse_rstfile_sexpdata(RSTDT_TEST_FILE)
# a list that contains Symbol instances (and lists of Symbol instances and integers)

In [11]:
root = sexp_tree[0]
    

In [12]:
print sexp_tree[1]
print sexp_tree[1][0]
print sexp_tree[1][0].value()

[Symbol('span'), 1, 47]
Symbol('span')
span


In [13]:
nuc_tree = sexp_tree[2]
print nuc_tree[1][0].value()
print nuc_tree[1][1], nuc_tree[1][2]
for i, e in enumerate(nuc_tree):
    print i, e, '\n'

span
1 20
0 Symbol('Nucleus') 

1 [Symbol('span'), 1, 20] 

2 [Symbol('rel2par'), Symbol('span')] 

3 [Symbol('Nucleus'), [Symbol('span'), 1, 14], [Symbol('rel2par'), Symbol('span')], [Symbol('Nucleus'), [Symbol('span'), 1, 8], [Symbol('rel2par'), Symbol('span')], [Symbol('Nucleus'), [Symbol('span'), 1, 4], [Symbol('rel2par'), Symbol('Inverted-Sequence')], [Symbol('Nucleus'), [Symbol('span'), 1, 3], [Symbol('rel2par'), Symbol('span')], [Symbol('Satellite'), [Symbol('leaf'), 1], [Symbol('rel2par'), Symbol('attribution')], [Symbol('text'), Symbol('_!Tandy'), Symbol('Corp.'), Symbol('said_!')]], [Symbol('Nucleus'), [Symbol('span'), 2, 3], [Symbol('rel2par'), Symbol('span')], [Symbol('Nucleus'), [Symbol('leaf'), 2], [Symbol('rel2par'), Symbol('span')], [Symbol('text'), Symbol('_!it'), Symbol('won'), Quoted(True), Symbol('join'), Symbol('U.S.'), Symbol('Memories,'), Symbol('the'), Symbol('group_!')]], [Symbol('Satellite'), [Symbol('leaf'), 3], [Symbol('rel2par'), Symbol('elaboration-object-

# SEXPDATA fail:  ' must be escaped

```python
>>> sexpdata.loads("(text this won't hurt)")
>>> [Symbol('text'), Symbol('this'), Symbol('won'), Quoted(True), Symbol('hurt')]
```

# Epic fail: RST-DT files contain superflous ``//TT_ERR`` strings
    
* I fixed the files in the ``RSTtrees-WSJ-main-1.0-tokenized`` directory

```
arne@ziegelstein ~/repos/rst_discourse_treebank $ ack-grep -cl "//TT_ERR"
data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_2367.out.dis:102
data/RSTtrees-WSJ-main-1.0/TRAINING/wsj_2353.out.dis:53
data/RSTtrees-WSJ-main-1.0-tokenized/TRAINING/wsj_2367.out.dis:102
data/RSTtrees-WSJ-main-1.0-tokenized/TRAINING/wsj_2353.out.dis:53
```

In [14]:
import discoursegraphs as dg
from collections import Counter

class RSTLispDocumentGraph(dg.DiscourseDocumentGraph):
    """
    A directed graph with multiple edges (based on a networkx
    MultiDiGraph) that represents the rhetorical structure of a
    document.

    Attributes
    ----------
    name : str
        name, ID of the document or file name of the input file
    ns : str
        the namespace of the document (default: rst)
    root : str
        name of the document root node ID
    tokens : list of str
        sorted list of all token node IDs contained in this document graph
    """
    def __init__(self, dis_filepath, name=None, namespace='rst',
                 tokenize=True, precedence=False):
        """
        Creates an RSTLispDocumentGraph from a Rhetorical Structure *.dis file and adds metadata
        to it.

        Parameters
        ----------
        dis_filepath : str
            absolute or relative path to the Rhetorical Structure *.dis file to be
            parsed.
        name : str or None
            the name or ID of the graph to be generated. If no name is
            given, the basename of the input file is used.
        namespace : str
            the namespace of the document (default: rst)
        precedence : bool
            If True, add precedence relation edges
            (root precedes token1, which precedes token2 etc.)
        """
        # super calls __init__() of base class DiscourseDocumentGraph
        super(RSTLispDocumentGraph, self).__init__()

        self.name = name if name else os.path.basename(dis_filepath)
        self.ns = namespace
        self.root = 0
        self.add_node(self.root, layers={self.ns}, label=self.ns+':root_node')
        if 'discoursegraph:root_node' in self:
            self.remove_node('discoursegraph:root_node')
        
        self.tokenized = tokenize
        self.tokens = []
        self.rst_tree = parse_rstfile_sexpdata(dis_filepath)
        self.parse_rst_tree(self.rst_tree)
        
    def parse_rst_tree(self, rst_tree, indent=0):
        tree_type = self.get_tree_type(rst_tree)
        assert tree_type in ('Root', 'Nucleus', 'Satellite')
        if tree_type == 'Root':
            span, children = rst_tree[1], rst_tree[2:]
            for child in children:
                self.parse_rst_tree(child, indent=indent+1)

        else: # tree_type in ('Nucleus', 'Satellite')
            node_id = self.get_node_id(rst_tree)
            node_type = self.get_node_type(rst_tree)
            relation_type = self.get_relation_type(rst_tree)
            if node_type == 'leaf':
                edu_text = self.get_edu_text(rst_tree[3])
                self.add_node(node_id, attr_dict={self.ns+':text': edu_text,
                                                  'label': u'{}: {}'.format(node_id, edu_text[:20])})
                if self.tokenized:
                    edu_tokens = edu_text.split()
                    for i, token in enumerate(edu_tokens):
                        token_node_id = '{}_{}'.format(node_id, i)
                        self.tokens.append(token_node_id)
                        self.add_node(token_node_id, attr_dict={self.ns+':token': token,
                                                                'label': token})
                        self.add_edge(node_id, '{}_{}'.format(node_id, i))
                    
            else: # node_type == 'span'
                self.add_node(node_id, attr_dict={self.ns+':rel_type': relation_type,
                                                   self.ns+':node_type': node_type})
                children = rst_tree[3:]
                child_types = self.get_child_types(children)
                
                expected_child_types = set(['Nucleus', 'Satellite'])
                unexpected_child_types = set(child_types).difference(expected_child_types)
                assert not unexpected_child_types, \
                    "Node '{}' contains unexpected child types: {}\n".format(node_id, unexpected_child_types)
                
                if 'Satellite' not in child_types:
                    # span only contains nucleii -> multinuc
                    for child in children:
                        child_node_id = self.get_node_id(child)
                        self.add_edge(node_id, child_node_id, attr_dict={self.ns+':rel_type': relation_type})
                
                elif len(child_types['Satellite']) == 1 and len(child_types['Nucleus']) == 1:
                    # standard RST relation, where one satellite is dominated by one nucleus
                    nucleus_index = child_types['Nucleus'][0]
                    satellite_index = child_types['Satellite'][0]
                    
                    nucleus_node_id = self.get_node_id(children[nucleus_index])
                    satellite_node_id = self.get_node_id(children[satellite_index])
                    self.add_edge(node_id, nucleus_node_id, attr_dict={self.ns+':rel_type': 'span'},
                                  edge_type=dg.EdgeTypes.spanning_relation)
                    self.add_edge(nucleus_node_id, satellite_node_id,
                                  attr_dict={self.ns+':rel_type': relation_type},
                                  edge_type=dg.EdgeTypes.dominance_relation)
                else:
                    raise ValueError("Unexpected child combinations: {}\n".format(child_types))
                
                for child in children:
                    self.parse_rst_tree(child, indent=indent+1)

    def get_child_types(self, children):
        """
        maps from (sub)tree type (i.e. Nucleus or Satellite) to a list
        of all children of this type
        """
        child_types = defaultdict(list)
        for i, child in enumerate(children):
            child_types[self.get_tree_type(child)].append(i)
        return child_types


    def get_edu_text(self, text_subtree):
        assert text_subtree[0].value() == 'text'
        return u' '.join(word.value().decode('utf-8')
                         if isinstance(word, sexpdata.Symbol) else word.decode('utf-8')
                         for word in text_subtree[1:])
        
    def get_tree_type(self, tree):
        """returns the type of the (sub)tree: Root, Nucleus or Satellite"""
        tree_type = tree[0].value()
        return tree_type

    def get_node_type(self, tree):
        """returns the node type (leaf or span) of a subtree (i.e. Nucleus or Satellite)"""
        node_type = tree[1][0].value()
        assert node_type in ('leaf', 'span')
        return node_type

    def get_relation_type(self, tree):
        """returns the RST relation type attached to the parent node of an RST relation"""
        return tree[2][1].value()

    def get_node_id(self, nuc_or_sat):
        node_type = self.get_node_type(nuc_or_sat)
        if node_type == 'leaf':
            leaf_id = nuc_or_sat[1][1]
            return '{}:{}'.format(self.ns, leaf_id)
        else: # node_type == 'span'
            span_start = nuc_or_sat[1][1]
            span_end = nuc_or_sat[1][2]
            return '{}:span:{}-{}'.format(self.ns, span_start, span_end)
        
        
    

Couldn't import dot_parser, loading of dot files will not be possible.


In [15]:
RSTDT_TOKENIZED_ROOT = os.path.expanduser('~/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-tokenized')

import traceback

# for folder in ('TEST', 'TRAINING'):
#     for rst_fpath in glob.glob(os.path.join(RSTDT_TOKENIZED_ROOT, folder, '*.dis')):
#         try:
#             RSTLispDocumentGraph(rst_fpath)
# #             print rst_fpath
#         except ValueError as e:
#             sys.stderr.write("Error in file '{}'\n{}\n".format(rst_fpath, e))


In [16]:
# TODO: error in attachment: rst:span:18-20 -> 18-19
rdg = RSTLispDocumentGraph(RSTDT_TOKENIZED_TEST_FILE, tokenize=False)

In [17]:
# %load_ext gvmagic
# %dotstr dg.print_dot(rdg)

# Do RSTDT-CoreNLP tokenizations match PTB?

In [18]:
RSTDT_NLTK_TOKENIZED_ROOT = os.path.expanduser('~/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized')

dis_file = os.path.join(RSTDT_NLTK_TOKENIZED_ROOT, 'TEST/wsj_2386.out.dis')
mrg_file = os.path.join(PTB_WSJ_ROOT_DIR, '23/wsj_2386.mrg')

rdg = RSTLispDocumentGraph(dis_file)
pdg = dg.read_ptb(mrg_file)

for t in rdg.tokens[:10]: print t,
print
for t in pdg.tokens[:10]: print t,

rst:1_0 rst:1_1 rst:1_2 rst:1_3 rst:1_4 rst:1_5 rst:1_6 rst:1_7 rst:1_8 rst:1_9
4 6 9 12 14 16 18 21 24 27


In [19]:
print dis_file
rdg.merge_graphs(pdg, verbose=True)

/home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_2386.out.dis


In [20]:
import re
import glob
import sys


WSJ_SUBDIR_REGEX = re.compile('wsj_(\d{2})')
WSJ_DOCID_REGEX = re.compile('wsj_(\d{4})')

for folder in ('TEST', 'TRAINING'):
    for rst_fpath in glob.glob(os.path.join(RSTDT_NLTK_TOKENIZED_ROOT, folder, '*.dis')):
        doc_id = os.path.basename(rst_fpath).split('.')[0]
        
        try:
            rdg = RSTLispDocumentGraph(rst_fpath)
            rst_fname = os.path.basename(rst_fpath).lower()

            doc_id = WSJ_DOCID_REGEX.match(rst_fname).groups()[0]
            wsj_subdir = WSJ_SUBDIR_REGEX.match(rst_fname).groups()[0]

            ptb_file = os.path.join(PTB_WSJ_ROOT_DIR, wsj_subdir, 'wsj_{}.mrg'.format(doc_id))
            pdg = dg.read_ptb(ptb_file)

            try:
                rdg.merge_graphs(pdg)
                print "merged: {}\n".format(rst_fpath)
            except Exception as e:
                sys.stderr.write("Error in {}: {}\n".format(rst_fpath, e))
        except Exception as e:
            sys.stderr.write("Error in {}: {}\n".format(rst_fpath, e))

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_2386.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_0627.out.dis


Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_2375.out.dis: 
Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_1387.out.dis: Tokenization mismatch: wsj_1387.out.dis (rst) vs. wsj_1387.mrg (ptb)
	Co. != Co



merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_0644.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_1365.out.dis


Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_0632.out.dis: Tokenization mismatch: wsj_0632.out.dis (rst) vs. wsj_0632.mrg (ptb)
	No != No.
Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_2385.out.dis: Tokenization mismatch: wsj_2385.out.dis (rst) vs. wsj_2385.mrg (ptb)
	B != B&H



merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_0667.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TEST/wsj_1189.out.dis


...

Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_1372.out.dis: Tokenization mismatch: wsj_1372.out.dis (rst) vs. wsj_1372.mrg (ptb)
	US != US$
Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_1368.out.dis: Tokenization mismatch: wsj_1368.out.dis (rst) vs. wsj_1368.mrg (ptb)
	S.p != S.p.A.



merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_1976.out.dis


Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_1388.out.dis: Tokenization mismatch: wsj_1388.out.dis (rst) vs. wsj_1388.mrg (ptb)
	. != ...
Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_1332.out.dis: Tokenization mismatch: wsj_1332.out.dis (rst) vs. wsj_1332.mrg (ptb)
	Bancorp != Bancorp.



merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_2341.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0699.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0647.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0615.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0613.out.dis

merged: /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0642.out.dis



Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0695.out.dis: Tokenization mismatch: wsj_0695.out.dis (rst) vs. wsj_0695.mrg (ptb)
	U.S != U.S.
Error in /home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0-nltk-tokenized/TRAINING/wsj_0680.out.dis: Tokenization mismatch: wsj_0680.out.dis (rst) vs. wsj_0680.mrg (ptb)
	a-Ex-dividend != a


In [21]:
os.path.basename(RSTDT_TEST_FILE)

'wsj_1306.out.dis'

In [22]:
PTB_TEST_FILE = os.path.expanduser('~/corpora/pennTreebank/parsed/mrg/wsj/13/wsj_1306.mrg')

In [23]:
sent0_root = pdg.sentences[0]
ptb_1306_tokens = list(pdg.get_tokens(token_strings_only=True))

# Epic Fail: we can't use nltk's Bracket parser, as it parses (span 1 5) as (span 1)

In [10]:
RSTDT_TEST_FILE

'/home/arne/repos/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TEST/wsj_1306.out.dis'

In [9]:
rst_tree = parse_rstfile_nltk(RSTDT_TEST_FILE)
span_tree = rst_tree[0]
print span_tree, span_tree.productions(), span_tree.leaves()


(span 1) [span -> '1'] [u'1']


In [21]:
print rst_tree[1][1]

(rel2par span)


In [16]:
# print open(RSTDT_TEST_FILE).read()